In [1]:
import numpy as np

data_x = np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])

data_sigma = np.array([[43584350.6], [43195861.8]])  


X_test=np.array([[0.014598, 0.014112], [0.014598, 0.016107131]])
y_test=np.array([[43584350.6], [43195861.8]]) 

print(data_x)
print(data_sigma)
assert data_x is not None, "data_x is None!"
assert data_sigma is not None, "data_sigma is None!"
print(type(data_x))  # Should print <class 'numpy.ndarray'>
print(type(data_sigma))  # Should print <class 'numpy.ndarray'>


[[0.014598   0.014112  ]
 [0.014598   0.01610713]]
[[43584350.6]
 [43195861.8]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [2]:
import deepxde as dde

# Use the DataSet data object to create a data-driven task
data = dde.data.dataset.DataSet(X_train=data_x, y_train=data_sigma, X_test=X_test, y_test=y_test)

# Define the network for σ
layer_size = [2] + [50] * 3 + [1]  # 2 inputs (x, y ) -> hidden layers -> 1 output (σ)
activation = "tanh"
initializer = "Glorot uniform"
net_sigma = dde.nn.FNN(layer_size, activation,initializer)

# Define the model
model_sigma = dde.Model(data, net_sigma)

# Compile the model with data
model_sigma.compile("adam", lr=1e-3)

# Train the model
losshistory, train_state = model_sigma.train(epochs=5000)


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.



Instructions for updating:
non-resource variables are not supported in the long term
Compiling model...
Building feed-forward neural network...

'build' took 0.160848 s




c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 0.269986 s

Training model...

Step      Train loss    Test loss     Test metric
0         [1.88e+15]    [1.88e+15]    []  
1000      [1.88e+15]    [1.88e+15]    []  
2000      [1.88e+15]    [1.88e+15]    []  
3000      [1.88e+15]    [1.88e+15]    []  
4000      [1.88e+15]    [1.88e+15]    []  
5000      [1.88e+15]    [1.88e+15]    []  

Best model at step 5000:
  train loss: 1.88e+15
  test loss: 1.88e+15
  test metric: []

'train' took 2.112179 s



In [118]:
sigma_A = model_sigma.predict([[2,1]])
print(sigma_A)

[[267.1882]]


In [12]:
import tensorflow as tf

def pde_C_L(x, C_L):
    dC_L_dt = dde.grad.jacobian(C_L, x, i=0, j=2)
    dC_L_dx2 = dde.grad.hessian(C_L, x, i=0, j=0)
    dC_L_dy2 = dde.grad.hessian(C_L, x, i=0, j=1)
    
    C_LA = C_L[:, 0:1]
    C_LB = C_L[:, 1:2]

    # Assuming data is stored in a 2D NumPy array, where columns represent [x, y, t]
    data = np.array(x)
    print(data)
    # Extract x, y pairs (assume column 0 is x, column 1 is y)
    x_y_pairs = data[:,0:2]
    print(x_y_pairs)
    # Randomly choose one row
    random_row = x_y_pairs[np.random.choice(x_y_pairs.shape[0])]
    print(random_row)    
    sigma_A = model_sigma.predict([random_row]) 
    print(sigma_A)

    

    eq2 = dC_L_dt - (1/3.8) * (dC_L_dx2 + dC_L_dy2)
    eq1 = C_LA - C_LB * tf.exp(2e10-6*(sigma_A))
   
    eq=eq1-eq2
    return [eq]






In [81]:
import tensorflow as tf
import math
def pde_C_L(x, C_L):
  
    C_LA = C_L[:, 0:1]
    C_LB = C_L[:, 1:2]

    # Assuming data is stored in a 2D NumPy array, where columns represent [x, y, t]
    data = np.array(x)
    print(data)
    # Extract x, y pairs (assume column 0 is x, column 1 is y)
    x_y_pairs = data[:,0:2]
    print(x_y_pairs)
    # Randomly choose one row
    #random_row = x_y_pairs[np.random.choice(x_y_pairs.shape[0])]
    #print(random_row)    
    sigma_A = model_sigma.predict(x_y_pairs) 
    print(sigma_A)
    eq1 = C_LA - C_LB *np.exp(sigma_A)
    return [eq1]


x = np.array([[1,1,1], [2,2,2], [3,3,3]])
C_L = np.array([[1,2], [3,4], [5,6]])  

print(pde_C_L(x, C_L))

[[1 1 1]
 [2 2 2]
 [3 3 3]]
[[1 1]
 [2 2]
 [3 3]]
[[267.765]
 [267.765]
 [267.765]]
[array([[-inf],
       [-inf],
       [-inf]])]


C:\Users\yesda\AppData\Local\Temp\ipykernel_37092\2224600891.py:19: RuntimeWarning: overflow encountered in exp
  eq1 = C_LA - C_LB *np.exp(sigma_A)


In [15]:
def pde_C_L(x, C_L):
    dC_L_dt = dde.grad.jacobian(C_L, x, i=0, j=2)
    dC_L_dx2 = dde.grad.hessian(C_L, x, i=0, j=0)
    dC_L_dy2 = dde.grad.hessian(C_L, x, i=0, j=1)

    eq2 = dC_L_dt - (1/3.8) * (dC_L_dx2 + dC_L_dy2)
    return [eq2]

In [42]:
x_y_pairs = np.array([[1, 2],
                      [3, 4],
                      [5, 6],
                      [7, 8],
                      [9, 10]])

# Shape will be (5, 2)
print(x_y_pairs.shape)  # Output: (5, 2)

# x_y_pairs.shape[0] gives the number of rows, which is 5
print(x_y_pairs.shape[0])  # Output: 5

# Select a random row
random_row = x_y_pairs[np.random.choice(x_y_pairs.shape[0])]
print(random_row)

(5, 2)
5
[5 6]


In [74]:
# Define the square domain (e.g., 2x2 square with side length L)
L = 1.0  # Half-length of the square plate
square = dde.geometry.Rectangle([-L, -L], [L, L])

# Define the circular hole at the center (radius R)
R = 0.1  # Radius of the hole
circle = dde.geometry.Disk([0, 0], R)

# Geometry and time domain
geom = dde.geometry.CSGDifference(square, circle)

timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
# Define the initial condition and boundary conditions
#ic = dde.icbc.IC(geomtime)
data = dde.data.TimePDE(
    geomtime,
    pde_C_L,
    [],
    num_domain=1000,
    num_boundary=150,
    num_initial=180,
    num_test=1000,
)

# Network for C_L
layer_size = [3] + [50] * 3 + [2]  # 3 inputs (x, y, t) -> hidden layers -> 2 outputs (C_LA, C_LB)
activation = "tanh"
initializer = "Glorot uniform"
net_C_L = dde.nn.FNN(layer_size, activation, initializer)



# Define the model with the PDE, initial and boundary conditions
model= dde.Model(data, net_C_L)

# Compile the model with the PDE and dataset
model.compile("adam", lr=1e-3)

# Train the model
model.train(epochs=5000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.029766 s



c:\Users\yesda\Documents\GitHub\PINN_testing\.conda\Lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


NotImplementedError: Cannot convert a symbolic tf.Tensor (Placeholder_62:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.

In [ ]:
# Assume C data is in the format: (x, y, z, t) -> C
data_C_L = np.hstack((data_x, t.reshape(-1, 1), data_C.reshape(-1, 1)))  # Combine x, y, z, t with C data

# Adding the C_L data as a point set boundary condition
observations_C_L = dde.PointSetBC(data_x, data_C, component=0)

# Add the C_L data to the model
model_C_L = dde.Model(geomtime, pde_C_L, [ic, observations_C_L], net_C_L)

# Train the model with both PDE and data-driven loss
model_C_L.compile("adam", lr=1e-3)
model_C_L.train(epochs=5000)


In [107]:
import deepxde as dde
import numpy as np


# Example data where each x and y have 3 components
X_train = np.array([[1.0], [2.0], [3.0]])  # Shape: (n_samples, 3)
y_train = np.array([[10.0], [11.0], [12.0]])  # Shape: (n_samples, 3)
# Ensure the data is not None and is correctly shaped
print("X_train:", X_train)
print("y_train:", y_train)

data = dde.data.dataset.DataSet(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)


# Define the neural network
net = dde.nn.FNN([1] + [50] * 3 + [1], "tanh", "Glorot uniform")


# Create and compile the model
model = dde.Model(data, net)
model.compile("adam", lr=0.001)
model.train(iterations=20000)
# Train the model
losshistory, train_state = model.train()


X_train: [[1.]
 [2.]
 [3.]]
y_train: [[10.]
 [11.]
 [12.]]
Compiling model...
Building feed-forward neural network...
'build' took 0.032669 s

'compile' took 0.255298 s

Training model...

0         [1.15e+02]    [6.23e-02]    []  
1000      [6.62e-02]    [4.88e+01]    []  
2000      [4.13e-02]    [5.77e+01]    []  
3000      [3.24e-03]    [7.02e+01]    []  
4000      [1.19e-07]    [7.36e+01]    []  
5000      [6.06e-13]    [7.36e+01]    []  
6000      [0.00e+00]    [7.36e+01]    []  
7000      [0.00e+00]    [7.36e+01]    []  


KeyboardInterrupt: 